In [30]:
import os

import numpy as np
import pandas as pd

# ---
run_prefix = "liming_noFert_hist_fixedRate_till"  # prefix of output run names
base_climdir = "/home/tykukla/aglime-swap-cdr/scepter/clim-inputs"
clim_general = "cesm2le_monthly_ens-1231-001"  # identifier for climate files to use
clim_ts = "1950-2020"
const_dict = {
    "duration": 70,  # duration of run (starts from earliest year)
    "dustsp": "cc",
    "dustrad": 150,
    # "nitrif": 10,    # [gN/m2/yr] (10 is ~ 90 lbs/acre/yr; 24.6 ~ 220 lbs/acre/year)
    "add_secondary": True,
    "climatedir": os.path.join(base_climdir, clim_general, clim_ts),
    "imix": 3,
}
# ---
savefn = run_prefix + "_v3-imix.csv"

In [31]:
# --- set the dust rate array
arr1 = np.arange(0, 1, 0.2)
arr2 = np.arange(1, 4, 1)
arr3 = np.arange(10, 40, 10)
dustflx_tonha = np.concatenate((arr1, arr2, arr3))
dustflx_in = dustflx_tonha * 100  # convert to g m-2
dustflx_in

array([   0.,   20.,   40.,   60.,   80.,  100.,  200.,  300., 1000.,
       2000., 3000.])

In [32]:
# --- read in site inputs
spin_in_path = "/home/tykukla/aglime-swap-cdr/scepter/batch-inputs"
spin_inputs = "spinup-inputs.csv"
# read in
df = pd.read_csv(os.path.join(spin_in_path, spin_inputs))
df

,site,spinname,lat,lon,mat,soilmoisture,qrun,tsom,erosion,nitrif,tph,cec,tec,tsoilco2,poro,alpha
0,site_311,site_311,42.5,-91,8.22219,0.282727,0.351361,2.051667,0.001013,1.005952,6.058007,21.10329,20.98031,-1.80371,0.447,2.0
1,site_411,site_411,32.0,-83,18.52789,0.231552,0.243426,2.276667,0.000840,0.831883,5.200242,1.96125,46.91557,-1.61194,0.419,2.0


In [33]:
# define the vars to collect
allvars = [
    "duration",
    "cec",
    "dustsp",
    "dustrate",
    "dustrad",
    "add_secondary",
    "spinrun",
    "imix",
    "newrun_id",
    "climatedir",
    "climatefiles",
]

In [34]:
def newrun_id_fxn(pref, site, clim_tag, dstflx):
    return pref + "_" + site + "_" + clim_tag + "_app-" + dstflx

In [35]:
# --- functions to derive inputs
def thisrow(tdf, const_dict, allvars, run_prefix, clim_ts, dstflx_fn, dstflx):
    outarr = []  # save row
    thissite = tdf["site"]
    for var in allvars:
        if var == "spinrun":
            outarr.append(thissite)
        elif var == "newrun_id":
            outarr.append(newrun_id_fxn(run_prefix, thissite, clim_ts, dstflx_fn))
        elif var == "climatefiles":
            outarr.append(thissite)
        else:  # var is defined by const_dict or df
            tvar = const_dict.get(var)
            if tvar is None:
                tvar = tdf[var]
                outarr.append(tvar)
            else:
                outarr.append(tvar)
    return outarr

In [36]:
# --- loop through sites
# create output pandas df
outdf = pd.DataFrame(columns=allvars)
# loop
cnt = 0
for rundx in range(len(dustflx_in)):
    for idx in range(len(df)):
        tdf = df.loc[idx]
        # get dust flux for filename and table
        dstflx_fn = str(dustflx_tonha[rundx].round(2)).replace(".", "p")
        dstflx = dustflx_in[rundx].round(2)
        # add to const dict
        const_dict.update({"dustrate": dstflx})
        thesevars = thisrow(
            tdf, const_dict, allvars, run_prefix, clim_ts, dstflx_fn, dstflx
        )
        outdf.loc[cnt] = thesevars
        cnt += 1
outdf

,duration,cec,dustsp,dustrate,dustrad,add_secondary,spinrun,imix,newrun_id,climatedir,climatefiles
0,70,21.10329,cc,0.0,150,True,site_311,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
1,70,1.96125,cc,0.0,150,True,site_411,3,liming_noFert_hist_fixedRate_till_site_411_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_411
2,70,21.10329,cc,20.0,150,True,site_311,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
3,70,1.96125,cc,20.0,150,True,site_411,3,liming_noFert_hist_fixedRate_till_site_411_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_411
4,70,21.10329,cc,40.0,150,True,site_311,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
5,70,1.96125,cc,40.0,150,True,site_411,3,liming_noFert_hist_fixedRate_till_site_411_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_411
6,70,21.10329,cc,60.0,150,True,site_311,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
7,70,1.96125,cc,60.0,150,True,site_411,3,liming_noFert_hist_fixedRate_till_site_411_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_411
8,70,21.10329,cc,80.0,150,True,site_311,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
9,70,1.96125,cc,80.0,150,True,site_411,3,liming_noFert_hist_fixedRate_till_site_411_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_411


In [37]:
# --- remove site 411 for now...
outdf = outdf.loc[outdf["climatefiles"] == "site_311"]
# update spinrun
outdf["spinrun"] = "site_311_pr8_spintuneup4"
outdf

/tmp/ipykernel_113663/3094738798.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outdf['spinrun'] = "site_311_pr8_spintuneup4"


,duration,cec,dustsp,dustrate,dustrad,add_secondary,spinrun,imix,newrun_id,climatedir,climatefiles
0,70,21.10329,cc,0.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
2,70,21.10329,cc,20.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
4,70,21.10329,cc,40.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
6,70,21.10329,cc,60.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
8,70,21.10329,cc,80.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
10,70,21.10329,cc,100.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
12,70,21.10329,cc,200.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
14,70,21.10329,cc,300.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
16,70,21.10329,cc,1000.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311
18,70,21.10329,cc,2000.0,150,True,site_311_pr8_spintuneup4,3,liming_noFert_hist_fixedRate_till_site_311_195...,/home/tykukla/aglime-swap-cdr/scepter/clim-inp...,site_311


In [38]:
outdf["newrun_id"][4]

'liming_noFert_hist_fixedRate_till_site_311_1950-2020_app-0p4'

In [39]:
# --- save result
savepath_batch = "/home/tykukla/aglime-swap-cdr/scepter/batch-inputs"
outdf.to_csv(os.path.join(savepath_batch, savefn), index=False)